In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.cluster import KMeans

In [2]:
trainx = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\new\\ready_train.csv')
trainy = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\new\\ready_y.csv')

In [3]:
#因為做了one hot encoding維度太高,所以用原始資料分群。
train = pd.read_csv('C:\\Users\\yunting\\Documents\\GitHub\\E.sun_conpetition\\data_set\\initial\\train.csv')

In [4]:
choose = [ 'town', 'village_income_median', 'total_price'] 

In [5]:
train['village_income_median'].describe()

count    58858.000000
mean       674.840175
std        140.653083
min        411.000000
25%        583.000000
50%        642.000000
75%        729.000000
max       1851.000000
Name: village_income_median, dtype: float64

In [6]:
train['village_income_median'].fillna(642,inplace = True)
train['village_income_median'].describe()

count    60000.000000
mean       674.215117
std        139.380337
min        411.000000
25%        584.000000
50%        642.000000
75%        728.000000
max       1851.000000
Name: village_income_median, dtype: float64

In [7]:
ks = KMeans(n_clusters=20).fit(train[choose])
labels = pd.DataFrame((ks.labels_).tolist())

In [8]:
labels.describe()

,0
count,60000.000000
mean,7.885217
std,8.312711
min,0.000000
25%,0.000000
50%,6.000000
75%,19.000000
max,19.000000


In [9]:
trainx_addlabels = pd.concat([trainx, labels, trainy],axis=1)
trainx_addlabels.head(3)

,III_10,III_100,III_1000,III_10000,III_250,III_50,III_500,III_5000,III_MIN,III_index_1000,...,vill_993,vill_994,vill_995,vill_996,vill_997,vill_998,vill_999,village_income_median,0,total_price
0,0,3,91,3728,17,1,37,1861,42.635964,1,...,0,0,0,0,0,0,0,572,0,6.476038e+05
1,1,1,29,7380,4,1,11,2512,7.157464,1,...,0,0,0,0,0,0,0,550,0,3.321452e+06
2,0,1,235,7971,17,0,73,2695,86.589412,1,...,0,0,0,0,0,0,0,625,19,9.570885e+06


# Group_10

In [10]:
group10 = trainx_addlabels[0]==10

In [11]:
group_10 = trainx_addlabels[group10]
group_10.head(3)

,III_10,III_100,III_1000,III_10000,III_250,III_50,III_500,III_5000,III_MIN,III_index_1000,...,vill_993,vill_994,vill_995,vill_996,vill_997,vill_998,vill_999,village_income_median,0,total_price
3,0,1,50,3904,3,1,5,2157,45.508654,1,...,0,0,0,0,0,0,0,950,10,14215011.13
9,0,2,201,10173,9,1,31,3849,36.215680,1,...,0,0,0,0,0,0,0,604,10,11815402.04
11,0,0,51,3925,4,0,17,2285,132.253089,1,...,0,0,0,0,0,0,0,950,10,11240596.77


In [12]:
Group_10 = group_10.drop([0], axis = 1)
Group_10.head(3)

,III_10,III_100,III_1000,III_10000,III_250,III_50,III_500,III_5000,III_MIN,III_index_1000,...,vill_992,vill_993,vill_994,vill_995,vill_996,vill_997,vill_998,vill_999,village_income_median,total_price
3,0,1,50,3904,3,1,5,2157,45.508654,1,...,0,0,0,0,0,0,0,0,950,14215011.13
9,0,2,201,10173,9,1,31,3849,36.215680,1,...,0,0,0,0,0,0,0,0,604,11815402.04
11,0,0,51,3925,4,0,17,2285,132.253089,1,...,0,0,0,0,0,0,0,0,950,11240596.77


## train Group_10

In [13]:
#標準化
norm_Group_10 = preprocessing.scale(Group_10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  


In [14]:
norm_Group_10_X = norm_Group_10[:, :-2] 
norm_Group_10_y = norm_Group_10[:, -1]

In [15]:
#CV
train_X , test_X , train_y, test_y = train_test_split (norm_Group_10_X , norm_Group_10_y, test_size = 0.3)
#KFold
kf = KFold(n_splits=5, shuffle=False)

# Decision Tree

In [16]:
from sklearn.tree import DecisionTreeRegressor

In [17]:
treereg = DecisionTreeRegressor()

## CV tree

In [18]:
cross_val_score(treereg, train_X, train_y, cv=10, n_jobs = -1)

array([0.66500105, 0.6461337 , 0.65063328, 0.65027631, 0.65253421,
       0.65394766, 0.66987912, 0.62548961, 0.68198672, 0.63890108])

In [20]:
treereg.fit(train_X, train_y)
treereg.score(test_X, test_y)

0.6889516086845446

## KFold tree

In [21]:
dscores = []
for train_index, test_index in kf.split(norm_Group_10_X):
    X_train, X_test, y_train, y_test = norm_Group_10_X[train_index], norm_Group_10_X[test_index], norm_Group_10_y[train_index], norm_Group_10_y[test_index]
    treereg.fit(X_train, y_train)
    dscores.append(treereg.score(X_test, y_test))

In [22]:
print('dscores: ', dscores)
print('means: ', np.mean(dscores))
print('vars:', np.var(dscores))

dscores:  [0.6993243967341346, 0.6871260320139343, 0.6763871011943139, 0.6737289639588134, 0.6661188465742705]
means:  0.6805370680950933
vars: 0.00013356715638932897


# xgboost

In [16]:
from xgboost import XGBRFRegressor

In [17]:
xgbfr = XGBRFRegressor()

## cv xgb

In [18]:
cross_val_score(xgbfr, train_X, train_y, cv=10, n_jobs = -1)

array([0.10815616, 0.06946711, 0.11421027, 0.09763328, 0.13816477,
       0.11979729, 0.11947955, 0.09703998, 0.09482505, 0.11275357])

## KFold xgb

In [19]:
xscores = []
for train_index, test_index in kf.split(norm_Group_10_X):
    X_train, X_test, y_train, y_test = norm_Group_10_X[train_index], norm_Group_10_X[test_index], norm_Group_10_y[train_index], norm_Group_10_y[test_index]
    xgbfr.fit(X_train, y_train)
    xscores.append(xgbfr.score(X_test, y_test))

[23:13:24] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:14:08] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:14:54] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:15:42] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:16:29] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
print('xscores: ', xscores)
print('means: ', np.mean(xscores))
print('vars:', np.var(xscores))

xscores:  [0.09822961067967284, 0.11008347700280663, 0.10826394686467311, 0.07887154216603554, 0.11174702723523211]
means:  0.10143912078968405
vars: 0.00014943052813283888
